In [4]:
import os
import pandas as pd
import numpy as np
import Data_Savior_J
from Data_Savior_J import load_file
from Data_Savior_J import save_it_now
import scipy as sp
from scipy import signal
from ml_time_series import generate_envelope

# Filtering

In [17]:
def filter_signal(emg, low_pass=10., sfreq=2000., high_band=20., low_band=450.):
    """
    emg: EMG data
    high: high-pass cut off frequency
    low: low-pass cut off frequency
    sfreq: sampling frequency
    """
    
    # normalise cut-off frequencies to sampling frequency
    high_band = high_band/(sfreq/2)
    low_band = low_band/(sfreq/2)
    
    # create bandpass filter for EMG
    b1, a1 = sp.signal.butter(4, [high_band,low_band], btype='bandpass')
    
    # process EMG signal: filter EMG
    emg_filtered = sp.signal.filtfilt(b1, a1, emg)    
    
    # process EMG signal: rectify
    emg_rectified = abs(emg_filtered)
    
    # create lowpass filter and apply to rectified signal to get EMG envelope
    low_pass = low_pass/sfreq
    b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')
    emg_envelope = sp.signal.filtfilt(b2, a2, emg_rectified)
    
    return emg_envelope

# Processing

In [18]:
def proccess_data(file_,DATA):
    
    #datafile = os.path.join(DATADIR, DATAFILE)
    #datafile = DATAFILE

    # Remove header, Nan and trash
    data = np.load('DB_HAND/'+file_+'.npy')
    data = filter_signal(data[:,0], low_pass=30)
    #plot_fremg(data)
    Xc_1 = data[:10000]
    Xc_2 = data[10000:20000]
    Xc_3 = data[20000:]
    
    print '\nXc_1 cleaned shape ', Xc_1.shape
    print 'Xc_2 cleaned shape ', Xc_2.shape
    print 'Xc_3 cleaned shape ', Xc_3.shape
    
    #Create temporal serie
    NUM_SAMPLES = 500
    
    Xc_1 = generate_envelope(Xc_1, NUM_SAMPLES)
    Xc_2 = generate_envelope(Xc_2, NUM_SAMPLES)
    Xc_3 = generate_envelope(Xc_3, NUM_SAMPLES)
    
    print 'Xc_1 temporal-serie shape ', Xc_1.shape
    print 'Xc_2 temporal-serie shape ', Xc_2.shape
    print 'Xc_3 temporal-serie shape ', Xc_3.shape
    
    #Labeling the type of movement
    C = (np.ones(len(Xc_1))*0).reshape((len(Xc_1),1))
    Xc_1 = np.hstack((Xc_1.reshape(Xc_1.shape),C.reshape((len(Xc_1),1))))


    C = (np.ones(len(Xc_2))*1).reshape((len(Xc_2),1))
    Xc_2 = np.hstack((Xc_2.reshape(Xc_2.shape),C.reshape((len(Xc_2),1))))
    
    C = (np.ones(len(Xc_3))*2).reshape((len(Xc_3),1))
    Xc_3 = np.hstack((Xc_3.reshape(Xc_3.shape),C.reshape((len(Xc_3),1))))
    
    print 'Xc_1 labeled shape ', Xc_1.shape
    print 'Xc_2 labeled shape ', Xc_2.shape
    print 'Xc_3 labeled shape ', Xc_3.shape
    
    # Salving in file on the folder <classifier_data>
    from Data_Savior_J import save_it_now
    save_it_now(Xc_1, './preproc_filtered_data/'+file_+'_f_honp.'+'data')
    print file_+'_o.'+'data'
    
    save_it_now(Xc_2, './preproc_filtered_data/'+file_+'_f_hclp.'+'data')
    print file_+'_cmf.'+'data'
    
    save_it_now(Xc_3, './preproc_filtered_data/'+file_+'_f_hchp.'+'data')
    print file_+'_ctf.'+'data'
    
    DATA.append(file_+'_f_honp.'+'data')
    DATA.append(file_+'_f_hclp.'+'data')
    DATA.append(file_+'_f_hchp.'+'data')

In [19]:
def proccess_data_validation(file_,DATA):
    
    #datafile = os.path.join(DATADIR, DATAFILE)
    #datafile = DATAFILE

    # Remove header, Nan and trash
    data = np.load('DB_HAND/'+file_+'.npy')
    data = filter_signal(data[:,0], low_pass=30)
    Xc_t = data
    print '\nXc_t cleaned shape ', Xc_t.shape
    
    #Create temporal serie
    NUM_SAMPLES = 500
    
    Xc_t = generate_envelope(Xc_t, NUM_SAMPLES)
    print 'Xc_t temporal-serie shape ', Xc_t.shape
    from Data_Savior_J import save_it_now
    save_it_now(Xc_t, './preproc_filtered_data/'+file_+'validation.'+'data')
    print file_+'validation.'+'data'
    
    DATA.append(file_+'validation.'+'data')

In [20]:
files = ['data_154743','data_155334','data_155743','data_161034', 'data_161420']
DATA = []
for i in files:
    proccess_data(i,DATA)


Xc_1 cleaned shape  (10000,)
Xc_2 cleaned shape  (10000,)
Xc_3 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9501, 500)
Xc_2 temporal-serie shape  (9501, 500)
Xc_3 temporal-serie shape  (9501, 500)
Xc_1 labeled shape  (9501, 501)
Xc_2 labeled shape  (9501, 501)
Xc_3 labeled shape  (9501, 501)
Saved to file
data_154743_o.data
Saved to file
data_154743_cmf.data
Saved to file
data_154743_ctf.data

Xc_1 cleaned shape  (10000,)
Xc_2 cleaned shape  (10000,)
Xc_3 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9501, 500)
Xc_2 temporal-serie shape  (9501, 500)
Xc_3 temporal-serie shape  (9501, 500)
Xc_1 labeled shape  (9501, 501)
Xc_2 labeled shape  (9501, 501)
Xc_3 labeled shape  (9501, 501)
Saved to file
data_155334_o.data
Saved to file
data_155334_cmf.data
Saved to file
data_155334_ctf.data

Xc_1 cleaned shape  (10000,)
Xc_2 cleaned shape  (10000,)
Xc_3 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9501, 500)
Xc_2 temporal-serie shape  (9501, 500)
Xc_3 temporal-serie shape 

In [5]:
#Validation set

Xvalid = load_file("preproc_filtered_data/rob_data_170159validation.data")
X_valid = Xvalid[:,:499]
X_valid.shape

(29501, 499)

In [6]:
Xc = load_file("preproc_filtered_data/Xc_set.data")


In [8]:
#Training and test set
Xc = load_file("preproc_filtered_data/"+DATA[0])
for i in DATA[1:]:
    Xc = np.vstack((Xc,load_file("preproc_filtered_data/"+i))) 

NameError: name 'DATA' is not defined

In [9]:
X = Xc[:,0:(Xc.shape[1]-2)]

yz = Xc[:,[(Xc.shape[1]-1)]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))

X.shape, y.shape

((142515, 499), (142515,))

In [10]:
from sklearn import model_selection
kfold = model_selection.ShuffleSplit(n_splits=2, test_size=.33, random_state=10)

In [11]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)

%time results = model_selection.cross_val_score(rfc, X, y, cv=kfold)

CPU times: user 8min 16s, sys: 444 ms, total: 8min 16s
Wall time: 8min 17s


In [12]:
%%bash
paplay /usr/share/sounds/freedesktop/stereo/complete.oga

In [13]:
print("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0)

Accuracy: 99.993% (0.007%)


In [22]:
x = ("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0)

In [37]:
str(x)

'Accuracy: 99.993% (0.007%)'

In [70]:
import smtplib
from email.MIMEMultipart import MIMEMultipart
from email.MIMEText import MIMEText

def send_email(text, toaddr='jorgeluizjk@gmail.com'):
    fromaddr = 'ohperaml@gmail.com'
    passw = 'ohperaml11'
    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = toaddr
    msg['Subject'] = "Jupyter - Processing completed"

    body = 'Processing completed:\n' + text
    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(fromaddr, passw)
    text = msg.as_string()
    server.sendmail(fromaddr, toaddr, text)
    server.quit()

In [71]:

send_email(x,'thaislealss@gmail.com')